In [ ]:
from imutils import contours
import numpy as np
import imutils
import cv2
import matplotlib.pyplot as plt
import math
%matplotlib inline  

In [ ]:
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect

def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    # return the warped image
    return warped

In [ ]:
image = cv2.imread("scan_test.jpg")
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image, height = 500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)

fig = plt.figure()
plt.imshow(edged, cmap="gray")

cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    # if our approximated contour has four points, then we
    # can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        break

fig = plt.figure()
top_left = screenCnt.reshape(4, 2)[0]
bottom_right = screenCnt.reshape(4, 2)[2]
bottom_left = screenCnt.reshape(4, 2)[1]
top_right = screenCnt.reshape(4, 2)[3]
print(screenCnt.reshape(4, 2))
plt.imshow(edged[top_left[1]: bottom_left[1],top_left[0]: top_right[0]], cmap="gray")
        
warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
thresh = cv2.adaptiveThreshold(warped, 255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
warped = (warped > thresh).astype("uint8") * 255

h, w = warped.shape
print(w, h)

if h > w:
    warped = cv2.rotate(warped, cv2.ROTATE_90_CLOCKWISE)

h, w = warped.shape
first_half = warped[:, 0:w//2]
second_half = warped[:, w//2:w]
first_half_density = np.sum(first_half)
second_half_density = np.sum(second_half)

if first_half_density < second_half_density:
    warped = cv2.rotate(warped, cv2.ROTATE_180)
    
warped = cv2.resize(warped, (500, 320))

h, w = warped.shape
top = int(h - 0.1091 * h)
bottom = int(h - 0.0203 * h)
start = int(0.1428 * w)
end = int(0.4418 * w)

iin = warped[top:bottom, start:end]

cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
fig = plt.figure()
plt.imshow(iin, cmap='gray')

In [ ]:
plt.imshow(warped, cmap='gray')

In [ ]:
cv2.imwrite("iin_test.jpg",  iin)